# Generating and Merging Isochrone Areas for Railway Stations Using OpenRouteService

## Step 1 – Batch Request and Export of Isochrone Areas from ORS

In [ ]:
import openrouteservice
import pandas as pd
import os
import time
import json

# Initialize ORS client
client = openrouteservice.Client(key='xxx')  

# Read station data
df = pd.read_csv('railway_stations.csv')
stations = df[['StationName', 'WGS84_Lng', 'WGS84_Lat']].dropna()

# Output directory and log file paths
output_folder = 'isochrones_geojson'
log_success_path = 'success_log.txt'
log_fail_path = 'fail_log.txt'

os.makedirs(output_folder, exist_ok=True)

# Track already processed files to avoid duplication
done_files = set([f.split('_')[0] for f in os.listdir(output_folder) if f.endswith('.geojson')])

# Limit the number of requests per run
MAX_RUN = 300
run_count = 0

# Open log files in append mode
with open(log_success_path, 'a', encoding='utf-8') as success_log, \
     open(log_fail_path, 'a', encoding='utf-8') as fail_log:

    for _, row in stations.iterrows():
        name = row['StationName']
        lon, lat = row['WGS84_Lng'], row['WGS84_Lat']

        if name in done_files:
            print(f'Skipped (already processed): {name}')
            continue

        try:
            result = client.isochrones(
                locations=[[lon, lat]],
                range=[3600],
                range_type='time',
                location_type='start',
                smoothing=0.3
            )

            # Save as GeoJSON
            out_path = os.path.join(output_folder, f'{name}_1hour.geojson')
            with open(out_path, 'w', encoding='utf-8') as f:
                json.dump(result, f)

            print(f'✅ Success: {name}')
            success_log.write(f'{name}\n')
            run_count += 1
            time.sleep(2)

            if run_count >= MAX_RUN:
                print(f'🔚 Max run limit reached ({MAX_RUN} records). Please continue tomorrow.')
                break

        except Exception as e:
            print(f'❌ Failed: {name}, Error: {e}')
            fail_log.write(f'{name},{e}\n')


## Step 2 – Merging Individual Isochrone GeoJSON Files into a Single Layer

In [ ]:
import geopandas as gpd
import os
import pandas as pd

geojson_folder = 'isochrones_geojson'
geojson_files = [os.path.join(geojson_folder, f) for f in os.listdir(geojson_folder) if f.endswith('.geojson')]

gdf_list = [gpd.read_file(f) for f in geojson_files]
merged_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
merged_gdf.crs = 'EPSG:4326'

merged_gdf.to_file('merged_isochrones.geojson', driver='GeoJSON')
print('✅ Merging completed. Output file: merged_isochrones.geojson')


# Batch Retrieval and Coordinate Conversion of Long-Distance Bus Stations in Guangdong via AMap API

In [ ]:
import requests
import json
import geopandas as gpd
from shapely.geometry import Point
import time
import math

API_KEY = "xxx"  
KEYWORD = "长途汽车站"
CITY = "Guangdong"
OUTPUT_FILE = "guangdong_bus_stations.geojson"

# Check if coordinates are outside China
def out_of_china(lon, lat):
    return not (72.004 <= lon <= 137.8347 and 0.8293 <= lat <= 55.8271)

# Latitude offset transformation
def transform_lat(lon, lat):
    ret = -100.0 + 2.0 * lon + 3.0 * lat + 0.2 * lat * lat + \
        0.1 * lon * lat + 0.2 * math.sqrt(abs(lon))
    ret += (20.0 * math.sin(6.0 * lon * math.pi) + 20.0 *
            math.sin(2.0 * lon * math.pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * math.pi) + 40.0 *
            math.sin(lat / 3.0 * math.pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * math.pi) + 320 *
            math.sin(lat * math.pi / 30.0)) * 2.0 / 3.0
    return ret

# Longitude offset transformation
def transform_lon(lon, lat):
    ret = 300.0 + lon + 2.0 * lat + 0.1 * lon * lon + \
        0.1 * lon * lat + 0.1 * math.sqrt(abs(lon))
    ret += (20.0 * math.sin(6.0 * lon * math.pi) + 20.0 *
            math.sin(2.0 * lon * math.pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lon * math.pi) + 40.0 *
            math.sin(lon / 3.0 * math.pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lon / 12.0 * math.pi) + 300.0 *
            math.sin(lon / 30.0 * math.pi)) * 2.0 / 3.0
    return ret

# Convert GCJ-02 to WGS-84
def gcj02_to_wgs84(lon, lat):
    if out_of_china(lon, lat):
        return lon, lat
    a = 6378245.0
    ee = 0.006693421622965943
    d_lat = transform_lat(lon - 105.0, lat - 35.0)
    d_lon = transform_lon(lon - 105.0, lat - 35.0)
    rad_lat = lat / 180.0 * math.pi
    magic = math.sin(rad_lat)
    magic = 1 - ee * magic * magic
    sqrt_magic = math.sqrt(magic)
    d_lat = (d_lat * 180.0) / ((a * (1 - ee)) / (magic * sqrt_magic) * math.pi)
    d_lon = (d_lon * 180.0) / (a / sqrt_magic * math.cos(rad_lat) * math.pi)
    return lon - d_lon, lat - d_lat

# Request POIs from AMap API
def fetch_poi(page=1):
    url = "https://restapi.amap.com/v3/place/text"
    params = {
        "key": API_KEY,
        "keywords": KEYWORD,
        "city": CITY,
        "output": "JSON",
        "offset": 50,
        "page": page,
        "extensions": "base"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def main():
    all_pois = []
    for page in range(1, 101):  # Up to 100 pages
        data = fetch_poi(page)
        if data and data.get("pois"):
            pois = data["pois"]
            if not pois:
                break
            all_pois.extend(pois)
            time.sleep(0.2)  # Avoid rate limiting
        else:
            break

    print(f"Total number of long-distance bus stations retrieved: {len(all_pois)}")

    records = []
    for poi in all_pois:
        try:
            lon_gcj, lat_gcj = map(float, poi["location"].split(","))
            lon_wgs, lat_wgs = gcj02_to_wgs84(lon_gcj, lat_gcj)
            records.append({
                "name": poi.get("name"),
                "address": poi.get("address"),
                "type": poi.get("type"),
                "gcj02_location": poi.get("location"),
                "geometry": Point(lon_wgs, lat_wgs)
            })
        except:
            continue

    gdf = gpd.GeoDataFrame(records, geometry="geometry", crs="EPSG:4326")
    gdf.to_file(OUTPUT_FILE, driver="GeoJSON")
    print(f"Exported to WGS84 GeoJSON file: {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Read the imported GeoJSON file
gdf = gpd.read_file("guangdong_bus_stations.geojson")

# Clean the 'address' field: ensure all values are strings to avoid map/null types
def clean_address(val):
    if isinstance(val, str):
        return val
    else:
        return ""

# Apply the cleaning function
gdf["address"] = gdf["address"].apply(clean_address)

# Export to a new GeoJSON file (cleaned data)
gdf.to_file("guangdong_bus_stations_clean.geojson", driver="GeoJSON")
print("Field cleaning completed. Exported to guangdong_bus_stations_clean.geojson.")


In [ ]:
import geopandas as gpd

# Read the cleaned GeoJSON file
gdf = gpd.read_file("guangdong_bus_stations_clean.geojson")

# Export to Shapefile format (a set of .shp/.shx/.dbf/.prj files will be created)
gdf.to_file("guangdong_bus_stations_clean.shp", driver="ESRI Shapefile")

print("Exported to Shapefile format: guangdong_bus_stations_clean.shp")
